In [1]:
from pyntcloud import PyntCloud
import pyntcloud
from pyntcloud.io import bin as io_bin
import numpy as np
import pandas as pd
import os

from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink

In [2]:
VELODYNE_HEIGHT = 1.73

def extractBB(label,gt=True):
    label_split = label.split(" ")
    if gt:
        return label_split[0],float(label_split[-7]),float(label_split[-6]),float(label_split[-5]),float(label_split[-4]),float(label_split[-3]),float(label_split[-2]),float(label_split[-1])
    else:
        return label_split[0],float(label_split[-8]),float(label_split[-7]),float(label_split[-6]),float(label_split[-5]),float(label_split[-4]),float(label_split[-3]),float(label_split[-2])
    
def getCorners(height,width,length,x,y,z,θ,rotation=True):
    
    corners = np.array([[-length / 2, -length / 2, length / 2, length / 2, -length / 2, -length / 2, length / 2, length / 2],
                        [width / 2, -width / 2, -width / 2, width / 2, width / 2, -width / 2, -width / 2, width / 2],
                        [0, 0, 0, 0, height, height, height, height]])
    
    rotMat = np.array([[np.cos(θ) , -np.sin(θ) , 0],
                       [np.sin(θ) ,  np.cos(θ) , 0],
                       [    0     ,     0      , 1]])
    if rotation:
        cornersPos = (np.dot(rotMat,corners)+np.tile([x,y,z],(8,1)).T).transpose()
        corner1,corner2,corner3,corner4,corner5,corner6,corner7,corner8 = cornersPos[0],cornersPos[1],cornersPos[2],cornersPos[3],cornersPos[4],cornersPos[5],cornersPos[6],cornersPos[7]
    else:
        cornersPos = (corners + np.tile([x,y,z],(8,1)).T).transpose()
        corner1,corner2,corner3,corner4,corner5,corner6,corner7,corner8 = cornersPos[0],cornersPos[1],cornersPos[2],cornersPos[3],cornersPos[4],cornersPos[5],cornersPos[6],cornersPos[7]
    
    return list(corner1),list(corner2),list(corner3),list(corner4),list(corner5),list(corner6),list(corner7),list(corner8)

def createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color="yellow"):
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C3,C4,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C4,C8,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C6,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C2,C6,C7,C3,C2]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C3,C7,C8,C4,C3]
        })
    return bounding_box

def load_label(file_name, label_list, text):
#     file_name = label_list[int(text)]
    labels = np.fromfile(file_name, dtype=np.int32).reshape((-1))    
    return labels

def hsv2rgb(h,s,v):
    import colorsys
    return list(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))
        
def hsbcolor(max_expected = 10, sat = 175, bri = 175):
    hue_factor = 255 / max_expected
    rgb_list = [hsv2rgb(((i*hue_factor)%255)/255, sat/255, bri/255) for i in range(max_expected)]
    color_list = np.array(rgb_list, dtype=np.uint8)
    return color_list
     

In [3]:
# red = np.array([255, 0, 0], dtype=np.uint8)
# green = np.array([0, 255, 0], dtype=np.uint8)
# blue = np.array([0, 0, 255], dtype=np.uint8)
# list0fpointindex = np.arange(500).tolist()
# classcolorlist = [green if i in list0fpointindex else red for i in range(len(new_pts))]

In [15]:
import glob 
label_list = np.sort(glob.glob("data/scene1/label/*.label"))
pc_list = np.sort(glob.glob("data/scene1/lidar/*.ply"))
label_list

array(['data/scene1/label\\000.label', 'data/scene1/label\\001.label',
       'data/scene1/label\\002.label', 'data/scene1/label\\003.label',
       'data/scene1/label\\004.label', 'data/scene1/label\\005.label',
       'data/scene1/label\\006.label', 'data/scene1/label\\007.label',
       'data/scene1/label\\008.label', 'data/scene1/label\\009.label',
       'data/scene1/label\\010.label', 'data/scene1/label\\011.label',
       'data/scene1/label\\012.label', 'data/scene1/label\\013.label',
       'data/scene1/label\\014.label', 'data/scene1/label\\015.label',
       'data/scene1/label\\016.label', 'data/scene1/label\\017.label',
       'data/scene1/label\\018.label', 'data/scene1/label\\019.label',
       'data/scene1/label\\020.label', 'data/scene1/label\\021.label',
       'data/scene1/label\\022.label', 'data/scene1/label\\023.label',
       'data/scene1/label\\024.label', 'data/scene1/label\\025.label',
       'data/scene1/label\\026.label', 'data/scene1/label\\027.label',
      

In [6]:
path= label_list[0]
#np.load(path, allow_pickle=True)
labels = np.fromfile(path, dtype=np.uint32).reshape(-1)
len(labels)

86185

In [12]:
labels

array([18, 18, 18, ..., 17, 18, 18], dtype=uint32)

In [8]:
path= pc_list[0]
points  = PyntCloud.from_file(path)
points.xyz.shape

(86185, 3)

In [9]:
new_pts = pd.DataFrame(points.xyz[:, 0:3], columns=['x', 'y', 'z'])
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(1.4262765645980835, 116.16038227081299, 59.3…

In [10]:
# Make seperate colors for segmentations
new_pts = pd.DataFrame(points.xyz[:, 0:3], columns=['x', 'y', 'z'])
list0fpointindex = np.fromfile(path, dtype=np.uint32).reshape(-1)
N = len(np.unique(list0fpointindex)) # Number of segmentation classes
colorlist = hsbcolor(N) # Color list
labelcolordict = {each:id for id,each in enumerate(np.unique(list0fpointindex))}
classcolorlist = [colorlist[labelcolordict[each]] for each in list0fpointindex[0:len(new_pts)]]

In [11]:
# Vizualize pointcloud with intensity
new_pts["red"]  = 0
new_pts["green"]  = 0
new_pts["blue"]  = 0
new_pts[["red","green", "blue"]] = classcolorlist
# new_pts.loc[list0fpointindex,["red","green", "blue"]] = blue
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(1.4262765645980835, 116.16038227081299, 59.3…

In [14]:
new_points = np.fromfile(path, dtype=np.float32)
new_points = new_points.reshape((-1, 3))
new_points = new_points[:, :3]

new_pts = pd.DataFrame(new_points[:, 0:3], columns=['x', 'y', 'z'])
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02)

ValueError: cannot reshape array of size 323237 into shape (3)